In [1]:
import tensorflow as tf

In [2]:
hello = tf.constant('Hello World')

In [3]:
type(hello)

tensorflow.python.framework.ops.Tensor

In [4]:
x = tf.constant(100)

In [5]:
type(x)

tensorflow.python.framework.ops.Tensor

In [9]:
import numpy as np
import tensorflow as tf
import pandas as pd
import time as time
import matplotlib.pyplot as plt
%matplotlib inline
print("TensorFlow version:",tf.__version__)

TensorFlow version: 1.14.0


In [10]:
data_dir ="/Earth_Quake_predict/data/"
model_dir ="/tmp/earth_quake_predict/models/1/"
summaries_dir = "/tmp/earth_quake_predict/logs"

In [29]:
#Extract data from CSV
df1 = pd.read_csv('C:/Users/WALI/Desktop/Earthquake_predict-master/earthquake_data.csv')
#df1=pd.read_csv(data_dir+ df2)
col1 = df1[['Latitude','Longitude','Depth']]   #Extract input data
col2 = df1['Magnitude']                     #Extract target data
#Convert to Numpy array
InputX1 = col1.as_matrix()
InputY1 = col2.as_matrix()
#Convert to float data type
InputX1.astype(float, copy=False)
InputY1.astype(float, copy=False)
#Determine shape of  input/output data
print("Shape of training input:",InputX1.shape)
print("Shape of values in training output:",InputY1.shape)
#Print input data
print("Input data:\n",InputX1)
#Print target data
print("Output data:\n",InputY1)
#Extract number of samples
batch_size = InputY1.shape[0]          #Number of training samples
#print(batch_size)

Shape of training input: (23412, 3)
Shape of values in training output: (23412,)
Input data:
 [[  19.246   145.616   131.6   ]
 [   1.863   127.352    80.    ]
 [ -20.579  -173.972    20.    ]
 ...
 [  36.9179  140.4262   10.    ]
 [  -9.0283  118.6639   79.    ]
 [  37.3973  141.4103   11.94  ]]
Output data:
 [6.  5.8 6.2 ... 5.9 6.3 5.5]


C:\Users\WALI\Anaconda3\envs\tfp3.6\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\WALI\Anaconda3\envs\tfp3.6\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [30]:
#Specify input and ouput features
Xfeatures = 3 #Number of input features
Yfeatures = 1 #Number of input features
#Reshape
InputX1 = np.resize(InputX1,(batch_size,Xfeatures))
InputY1 = np.resize(InputY1,(batch_size,Yfeatures))
#Specify size of training batch and validation batch
train_batch_size = 20000
valid_batch_size = batch_size-train_batch_size
print("Shape of training input:",InputX1.shape)
print("Shape of values in training output:",InputY1.shape)

Shape of training input: (23412, 3)
Shape of values in training output: (23412, 1)


In [31]:
# reset everything to rerun in jupyter
tf.reset_default_graph()
with tf.device('/cpu:0'):
    #Raw data input
    #raw_data = tf.placeholder(tf.float32,shape=(None),name='Raw_data')#[batch size, input_features]
    #Input variable
    X = tf.placeholder(tf.float32,shape=(None,Xfeatures),name='Input')#[batch size, input_features]
    #Output variable
    Y = tf.placeholder(tf.float32,name='Output')

    with tf.name_scope('Normalization'):
        #L2 normalization for input data
        Xnorm = tf.nn.l2_normalize(X, 0, epsilon=1e-12, name='Normalize')
    with tf.name_scope('Split_raw_data_set_into_training_and_validation'):
        #Slicing data set
        Xtrain = tf.slice(Xnorm,[0,0] ,[train_batch_size,-1], name='Extract_training_data_X')
        Ytrain = tf.slice(Y,[0,0] ,[train_batch_size,-1], name='Extract_training_data_Y')
        Xv = tf.slice(Xnorm,[train_batch_size,0] ,[valid_batch_size,-1], name='Extract_validation_data_X')
        Yv = tf.slice(Y,[train_batch_size,0] ,[valid_batch_size,-1], name='Extract_validation_data_Y')

In [32]:

##Network hyper parametres
L1 = 3 #Number of neurons in 1st layer
L2 = 3 #Number of neurons in 2nd layer
L3 = 3 #Number of neurons in 3rd layer
#Standard deviation of initial weights and biases
stddev0 = 0.3
with tf.device('/cpu:0'):
    with tf.name_scope('FeedForward_layer_1'):
        #Layer1 weights
        W_fc1 = tf.Variable(tf.random_normal([Xfeatures,L1],stddev=stddev0),name="FeedForward_layer_1_weights") # [input_features,Number of neurons]) 
        b_fc1 = tf.Variable(tf.random_normal([L1],stddev=stddev0),name="FeedForward_layer_1_bias")
        #Layer 1
        matmul_fc1=tf.matmul(X, W_fc1) + b_fc1
        h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation
        summ_W_fc1 = tf.summary.histogram("FC_Layer_1_Weights",W_fc1)
        summ_b_fc1 = tf.summary.histogram("FC_Layer_1_biases",b_fc1)
        
    with tf.name_scope('FeedForward_layer_2'):
        #Layer2 weights
        W_fc2 = tf.Variable(tf.random_normal([L1,L2],stddev=stddev0),name="FeedForward_layer_2_weights") # [Number of neurons in preceding layer,Number of neurons]) 
        b_fc2 = tf.Variable(tf.random_normal([L2],stddev=stddev0),name="FeedForward_layer_2_bias")
        #Layer 2
        matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
        h_fc2 = tf.nn.relu(matmul_fc2)   #ReLU activation
        summ_W_fc2 = tf.summary.histogram("FC_Layer_2_Weights",W_fc2)
        summ_b_fc2 = tf.summary.histogram("FC_Layer_2_biases",b_fc2)
    
    with tf.name_scope('FeedForward_layer_3'):
        #Layer3 weights
        W_fc3 = tf.Variable(tf.random_normal([L2,L3],stddev=stddev0),name="FeedForward_layer_3_weights") # [Number of neurons in preceding layer,Number of neurons]) 
        b_fc3 = tf.Variable(tf.random_normal([L3],stddev=stddev0),name="FeedForward_layer_3_bias")
        #Layer 3
        matmul_fc3=tf.matmul(h_fc2, W_fc3) + b_fc3
        h_fc3 = tf.nn.relu(matmul_fc3)   #ReLU activation
        summ_W_fc3 = tf.summary.histogram("FC_Layer_3_Weights",W_fc3)
        summ_b_fc3 = tf.summary.histogram("FC_Layer_3_biases",b_fc3)

    with tf.name_scope('Output_layer'):
        #Output layer weights
        W_fcO = tf.Variable(tf.random_normal([L3,Yfeatures],stddev=stddev0),name="Output_layer_weights") #  [Number of neurons in preceding layer,output_features]) 
        b_fcO = tf.Variable(tf.random_normal([Yfeatures],stddev=stddev0),name="Output_layer_bias")
        #Output layer
        matmul_fc4=tf.matmul(h_fc3, W_fcO) + b_fcO
        output_layer = matmul_fc4  #linear activation
        summ_W_fcO = tf.summary.histogram("Output_Layer_Weights",W_fcO)
        summ_b_fcO = tf.summary.histogram("Output_Layer_biases",b_fcO)

In [33]:
#Training hyper parametres
learning_rate = 0.00001
training_epochs = 1000000
display_epoch = 200000
log_epoch = 5000

In [34]:
with tf.device('/cpu:0'):
    with tf.name_scope('Loss_Function'):
        #Loss function
        #mean_square_error =  tf.reduce_mean(tf.square(Y-output_layer))
        mean_square_error = tf.losses.mean_squared_error(Y,output_layer)
        summ_train_loss= tf.summary.scalar('sigmoid_cross_entropy_train', mean_square_error)
        summ_valid_loss= tf.summary.scalar('sigmoid_cross_entropy_valid', mean_square_error)
        
    with tf.name_scope('Optimizer'):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square_error)
        #train_step = tf.train.AdagradOptimizer(learning_rate).minimize(mean_square_error)
        #train_step = tf.train.MomentumOptimizer(learning_rate,0.01).minimize(mean_square_error)
        #train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(mean_square_error)
        
    merged =tf.summary.merge([summ_train_loss,summ_W_fc1,summ_b_fc1,summ_W_fc2,summ_b_fc2,summ_W_fc3,summ_b_fc3,
                             summ_W_fcO,summ_b_fcO]) 
    valid_merged =tf.summary.merge([summ_valid_loss])

W0622 16:56:40.880215 67136 deprecation.py:323] From C:\Users\WALI\Anaconda3\envs\tfp3.6\lib\site-packages\tensorflow\python\ops\losses\losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [38]:
#Operation to save variables
saver = tf.train.Saver()
#Initialization and session
init = tf.global_variables_initializer()
log_iterations = 0
tstart = time.time()
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(summaries_dir+"/1/",sess.graph)  #For writing summaries
    sess.run(init)
    
    #Normalize training data
    #InputX1_norm = sess.run(normalize_raw_data,feed_dict={raw_data:InputX1})
    #InputY1_norm = sess.run(normalize_raw_data,feed_dict={raw_data:InputY1})
   
    
    InputX1train = sess.run(Xtrain,feed_dict={X:InputX1})
    InputY1train = sess.run(Ytrain,feed_dict={Y:InputY1})
    InputX1v = sess.run(Xv,feed_dict={X:InputX1})
    InputY1v = sess.run(Yv,feed_dict={Y:InputY1})
    
    print("Normalized training input:\n",InputX1train)
    print("Normalized training output:\n",InputY1train)
    print("Validation input:\n",InputX1v)
    print("Validation output:\n",InputY1v)
    
    print("Initial training loss:",sess.run(mean_square_error,feed_dict={X:InputX1train,Y:InputY1train}))
    print("Initial validation loss:",sess.run([mean_square_error],feed_dict={X:InputX1v,Y:InputY1v}))
        
    for i in range(training_epochs):
        sess.run([train_step],feed_dict={X:InputX1train,Y:InputY1train})
        if i%display_epoch ==0:
            print("Training loss is:",sess.run([mean_square_error],feed_dict={X:InputX1train,Y:InputY1train}),"at itertion:",i)
            print("Validation loss is:",sess.run([mean_square_error],feed_dict={X:InputX1v,Y:InputY1v}),"at itertion:",i)
        if i%log_epoch ==0:
            summary,_ = sess.run([merged,mean_square_error],feed_dict={X:InputX1train,Y:InputY1train})
            train_writer.add_summary(summary, log_iterations)
            valid_summary,_ = sess.run([valid_merged,mean_square_error],feed_dict={X:InputX1v,Y:InputY1v})
            train_writer.add_summary(valid_summary, log_iterations)
            log_iterations = log_iterations +1
            
    #Close summary writer
    train_writer.close()
    # Save the variables to disk.
    save_path = saver.save(sess, model_dir+"earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("Final training loss:",sess.run([mean_square_error],feed_dict={X:InputX1train,Y:InputY1train}))
    print("Final validation loss:",sess.run([mean_square_error],feed_dict={X:InputX1v,Y:InputY1v}))
    Ypredict = sess.run(output_layer,feed_dict={X:InputX1v})
    print(Ypredict)
print("Time taken:", (time.time()-tstart)/60)
print("Number of summaries:",log_iterations)

Normalized training input:
 [[ 0.00417061  0.00723048  0.00607405]
 [ 0.00040371  0.00632359  0.00369243]
 [-0.00445947 -0.00863848  0.00092311]
 ...
 [-0.00750086 -0.00357462  0.00161544]
 [-0.00746966 -0.00357532  0.00145389]
 [-0.00745037 -0.00357408  0.00036924]]
Normalized training output:
 [[6. ]
 [5.8]
 [6.2]
 ...
 [5.6]
 [6. ]
 [5.6]]
Validation input:
 [[-0.0074183  -0.003571    0.00032309]
 [ 0.00499754  0.00469845  0.00471247]
 [ 0.00047501  0.00660504  0.00046155]
 ...
 [ 0.00800012  0.00697279  0.00046155]
 [-0.00195643  0.00589219  0.00364628]
 [ 0.008104    0.00702165  0.0005511 ]]
Validation output:
 [[5.8]
 [5.5]
 [5.9]
 ...
 [5.9]
 [6.3]
 [5.5]]
Initial training loss: 38.401375
Initial validation loss: [38.474117]
Training loss is: [38.401196] at itertion: 0
Validation loss is: [38.47388] at itertion: 0
Training loss is: [17.233084] at itertion: 200000
Validation loss is: [17.287056] at itertion: 200000
Training loss is: [4.701739] at itertion: 400000
Validation loss 

ValueError: Parent directory of /tmp/earth_quake_predict/models/1/earthquake_model.ckpt doesn't exist, can't save.

In [ ]:
#Testing
df2=pd.read_csv('C:/Users/WALI/Desktop/Earthquake_predict-master/earthquake_data.csv')
col1 = df2[['Latitude','Longitude','Depth']]   #Extract input data
col2 = df1['Magnitude']   
#Convert to Numpy array
InputX = col1.as_matrix()
InputY = col2.as_matrix()

#Convert to float data type
InputX.astype(float, copy=False)
InputY.astype(float, copy=False)
#Determine shape of  test input data
print("Shape of testing input:",InputX.shape)
print("Shape of testing input:",InputY.shape)
test_batch_size = InputY.shape[0]          #Number of testing samples
#Reshape
InputX = np.resize(InputX,(test_batch_size,Xfeatures))
InputY = np.resize(InputY,(test_batch_size,Yfeatures))